In [1]:
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
import sklearn.metrics
import autosklearn.regression
import autosklearn.classification
import pickle
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import os
import pandas as pd
import warnings

In [2]:
data_test=pd.read_csv("testdata_2500.csv")

train_list = ['BBD_with_339']

mark = 5

result = []

for name in train_list:
    
    train_name = 'train_data1/{}.csv'.format(name)
    model_name = name

    data_train=pd.read_csv(train_name)  

    X_train=data_train.drop(columns=[" Displacement"])
    y_train=data_train[" Displacement"]

    X_test=data_test.drop(columns=[" Displacement"])
    y_test=data_test[" Displacement"]
    X_test,y_test

    print('training...')
    automl = autosklearn.regression.AutoSklearnRegressor(
        time_left_for_this_task=700, #? 600-700
        per_run_time_limit=70, # ?
        initial_configurations_via_metalearning=25,
        memory_limit=20480, 
        resampling_strategy='cv',
        resampling_strategy_arguments={'folds':5},
        n_jobs=18
    )
    automl.fit(X_train, y_train, dataset_name=model_name)
    print('training successfully')
#     print(automl.sprint_statistics())
    filename = 'aml_{}_{}.sav'.format(model_name, mark) 
    pickle.dump(automl, open('model/{}'.format(filename), 'wb'))
    print('saved')


    y_pred = automl.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = sklearn.metrics.r2_score(y_test, y_pred)
    
    line = [mark, name, r2, mae]
    print(line)

    result.append(line)
print(result)

training...


[WARNING] [2023-07-05 13:16:34,448:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-07-05 13:16:34,544:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-07-05 13:16:34,922:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-07-05 13:16:35,131:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-07-05 13:16:35,278:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-07-05 13:16:35,580:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-07-05 13:16:35,669:Client-EnsembleBuilder] No runs were available to build an ensemble from
training successfully
saved
[5, 'BBD_with_339', -0.0010912763127539638, 20.654095109981903]
[[5, 'BBD_with_339', -0.0010912763127539638, 20.654095109981903]]


## BBD
[[1, 'BBD_with_113', 0.7912394647501361, 7.904780815965377], [1, 'BBD_with_226', 0.8895890249653196, 6.545336554601771], [1, 'BBD_with_339', 0.7404107023284128, 11.661966702760104]]

[2, 'BBD_with_339', 0.10590475221607376, 15.378454444106174]
[[2, 'BBD_with_113', 0.7085871849384372, 10.381865710930649], [2, 'BBD_with_226', 0.891666972976674, 6.460148173883005], [2, 'BBD_with_339', 0.10590475221607376, 15.378454444106174]]

[[3, 'BBD_with_113', 0.7314590421402551, 10.751602084884771], [3, 'BBD_with_226', 0.894860758200982, 6.44113005178229], [3, 'BBD_with_339', 0.5136904259779761, 14.4662714956586]]

[[4, 'BBD_with_113', 0.6660824988981597, 11.35482921715777], [4, 'BBD_with_226', 0.8823799879908316, 6.5708053477637165], [4, 'BBD_with_339', 0.3012633906433706, 17.850639456684714]]

[[5, 'BBD_with_113', 0.6660824988981597, 11.35482921715777], [5, 'BBD_with_226', 0.8928554628113294, 6.525574754169677], [5, 'BBD_with_339', -0.30792775362559355, 23.057122727576544]]

## BBD339_2
[[1, 'BBD_with_339', 0.7470530689487631, 11.44804248803327]]
[[2, 'BBD_with_339', 0.3065597484575351, 13.78640917794698]]
[[3, 'BBD_with_339', 0.20082447749897192, 14.635566890143975]]
[[4, 'BBD_with_339', 0.15910623352119735, 19.28993519842574]]
[[5, 'BBD_with_339', 0.3570032073981999, 17.36233047457153]]

## 40 60 100
[[1, 'LH_with_40_c', 0.7459673276315595, 8.379741438589548], [1, 'LH_with_60_c', 0.8942535447078467, 4.867463571960389], [1, 'LH_with_100_c', 0.9025991765058894, 3.950968344595841], [1, 'LH_with_40_m', 0.7075368881863849, 7.384851767597199], [1, 'LH_with_60_m', 0.8113981314959697, 7.248237417262618], [1, 'LH_with_100_m', 0.7786773669166003, 4.974265609334761], [1, 'LH_with_40_cm', 0.674811393928142, 8.172708882633923], [1, 'LH_with_60_cm', 0.8095579602477125, 8.02628422065588], [1, 'LH_with_100_cm', 0.8991134318301799, 3.968035156752073], [1, 'EMU_20LH_20IT', 0.813591324077329, 9.381996317107157], [1, 'EMU_30LH_30IT', 0.8564578328275791, 6.431619845182256], [1, 'EMU_50LH_50IT', 0.7842467030569262, 13.192650979577738]]

[[2, 'LH_with_40_c', 0.7212074281434719, 8.28803032486073], [2, 'LH_with_60_c', 0.8942535447078467, 4.867463571960389], [2, 'LH_with_100_c', 0.8918205239733056, 4.425966432335987], [2, 'LH_with_40_m', 0.6948229958406509, 7.554977410146294], [2, 'LH_with_60_m', 0.8427219880847414, 6.251800429864951], [2, 'LH_with_100_m', 0.7771454981930088, 4.951705080929074], [2, 'LH_with_40_cm', 0.739945363510405, 6.580527190780078], [2, 'LH_with_60_cm', 0.7606513673719978, 8.764676758935297], [2, 'LH_with_100_cm', 0.9440938705874855, 3.0587866478518153], [2, 'EMU_20LH_20IT', 0.8595358995861513, 6.013456814828631], [2, 'EMU_30LH_30IT', 0.783613185602538, 9.014597732518395], [2, 'EMU_50LH_50IT', 0.9433008646472815, 4.757840808050331]]

[[3, 'LH_with_40_c', 0.8396951951496104, 5.508710422543823], [3, 'LH_with_60_c', 0.8010850203506569, 7.971838259117052], [3, 'LH_with_100_c', 0.8852444925888564, 4.804450872030992], [3, 'LH_with_40_m', 0.6871426150221192, 7.571749659161957], [3, 'LH_with_60_m', 0.721028617070271, 7.478797370489998], [3, 'LH_with_100_m', 0.7812250043955026, 4.97479542893685], [3, 'LH_with_40_cm', 0.7767304483245587, 6.3032352432468475], [3, 'LH_with_60_cm', 0.7931573253411064, 8.613582465891911], [3, 'LH_with_100_cm', 0.9435566261107963, 3.0216224860008967], [3, 'EMU_20LH_20IT', 0.7417551863310661, 7.725692151660954], [3, 'EMU_30LH_30IT', 0.7923587054557328, 8.843509357241844], [3, 'EMU_50LH_50IT', 0.9063700912693022, 5.705551887281942]]

[[4, 'LH_with_40_c', 0.715034774845975, 8.486256680708339], [4, 'LH_with_60_c', 0.7317856800914307, 9.267564098754214], [4, 'LH_with_100_c', 0.9120219624580316, 3.526294929402467], [4, 'LH_with_40_m', 0.6979561197280075, 7.29518199571571], [4, 'LH_with_60_m', 0.8229092472827813, 6.657029228435541], [4, 'LH_with_100_m', 0.781465399199901, 4.918117402858504], [4, 'LH_with_40_cm', 0.7154743376261221, 7.131684216642905], [4, 'LH_with_60_cm', 0.8140948282964267, 7.914315887857019], [4, 'LH_with_100_cm', 0.8980371233431371, 4.350704644663736], [4, 'EMU_20LH_20IT', 0.8595358995861513, 6.013456814828631], [4, 'EMU_30LH_30IT', 0.7950202096764195, 8.78217479040418], [4, 'EMU_50LH_50IT', 0.8202279895830157, 7.703514691640378]]

## 80
[[1, 'EMU_40LH_40IT', 0.7782762849394829, 11.135231977067276], [1, 'LH_with_80_c', 0.8002961427777011, 5.762781952575901], [1, 'LH_with_80_m', 0.8748675907559504, 4.863411741936946], [1, 'LH_with_80_cm', 0.707535161504822, 6.828762423168524]]

[[2, 'EMU_40LH_40IT', 0.9213850553555751, 6.012038712676582], [2, 'LH_with_80_c', 0.8197717854148204, 5.4445883029914155], [2, 'LH_with_80_m', 0.8545317050486337, 5.1482285290036325], [2, 'LH_with_80_cm', 0.7196714211144715, 6.747165742510803]]

[[3, 'EMU_40LH_40IT', 0.8632464364777541, 7.323685877735622], [3, 'LH_with_80_c', 0.8197187061236756, 5.446820186867361], [3, 'LH_with_80_m', 0.9072070523292997, 4.452280843233296], [3, 'LH_with_80_cm', 0.7179078489530637, 6.278312403176325]]

[[4, 'EMU_40LH_40IT', 0.8617277189534297, 7.184919108311076], [4, 'LH_with_80_c', 0.7768151934223807, 6.04950795359266], [4, 'LH_with_80_m', 0.8557899902712556, 5.2736617547766445], [4, 'LH_with_80_cm', 0.7118198659035503, 6.4878311066265315]]

[[5, 'EMU_40LH_40IT', 0.7863541452855694, 11.249325404243107], [5, 'LH_with_80_c', 0.8125895809240906, 5.597338481519742], [5, 'LH_with_80_m', 0.8602864265726738, 5.176139061596424], [5, 'LH_with_80_cm', 0.7671476314558864, 6.128248772884081]]

[[6, 'EMU_40LH_40IT', 0.8567811616379536, 6.908184380874815], [6, 'LH_with_80_c', 0.8108553152504032, 5.718217899836546], [6, 'LH_with_80_m', 0.8569521951686355, 5.288928483619603], [6, 'LH_with_80_cm', 0.717905430192268, 6.331164498620706]]

[[7, 'EMU_40LH_40IT', 0.7953667772603861, 8.175815940856365], [7, 'LH_with_80_c', 0.8070513914353031, 5.3619827701397424], [7, 'LH_with_80_m', 0.8856121230908417, 4.67979560022511], [7, 'LH_with_80_cm', 0.6497757930429913, 7.54973156557504]]

In [ ]:
'LH_with_120_c',
'LH_with_150_c',
'LH_with_180_c',
'LH_with_220_c',
'LH_with_250_c',
'LH_with_280_c',
'LH_with_320_c',
'LH_with_120_m',
'LH_with_150_m',
'LH_with_180_m',
'LH_with_220_m',
'LH_with_250_m',
'LH_with_280_m',
'LH_with_320_m',
'LH_with_120_cm',
'LH_with_150_cm',
'LH_with_180_cm',
'LH_with_220_cm',
'LH_with_250_cm',
'LH_with_280_cm',
'LH_with_320_cm',
'EMU_60LH_60IT',
'EMU_75LH_75IT',
'EMU_90LH_90IT',
'EMU_110LH_110IT',
'EMU_125LH_125IT',
'EMU_140LH_140IT',
'EMU_160LH_160IT'


'EMU_40LH_40IT',
'LH_with_80_c',
'LH_with_80_m',
'LH_with_80_cm',

'LH_with_40_c',
'LH_with_60_c',
'LH_with_100_c',
'LH_with_40_m',
'LH_with_60_m',
'LH_with_100_m',
'LH_with_40_cm',
'LH_with_60_cm',
'LH_with_100_cm',
'EMU_20LH_20IT',
'EMU_30LH_30IT',
'EMU_50LH_50IT',

'BBD_with_113','BBD_with_226','BBD_with_339'

## Result

[1, 'LH_with_120_c', 0.9482331129841547, 3.271069600657848]

6.20
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_c
  Metric: r2
  Best validation score: 0.980753
  Number of target algorithm runs: 157
  Number of successful target algorithm runs: 136
  Number of crashed target algorithm runs: 19
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_c
  Metric: r2
  Best validation score: 0.958130
  Number of target algorithm runs: 108
  Number of successful target algorithm runs: 95
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_c
  Metric: r2
  Best validation score: 0.881763
  Number of target algorithm runs: 83
  Number of successful target algorithm runs: 71
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_c
  Metric: r2
  Best validation score: 0.979468
  Number of target algorithm runs: 86
  Number of successful target algorithm runs: 70
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_c
  Metric: r2
  Best validation score: 0.985672
  Number of target algorithm runs: 74
  Number of successful target algorithm runs: 61
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_c
  Metric: r2
  Best validation score: 0.965434
  Number of target algorithm runs: 86
  Number of successful target algorithm runs: 78
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_c
  Metric: r2
  Best validation score: 0.984684
  Number of target algorithm runs: 95
  Number of successful target algorithm runs: 83
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_m
  Metric: r2
  Best validation score: 0.939029
  Number of target algorithm runs: 137
  Number of successful target algorithm runs: 122
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_m
  Metric: r2
  Best validation score: 0.979493
  Number of target algorithm runs: 122
  Number of successful target algorithm runs: 110
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_m
  Metric: r2
  Best validation score: 0.956798
  Number of target algorithm runs: 97
  Number of successful target algorithm runs: 84
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_m
  Metric: r2
  Best validation score: 0.908268
  Number of target algorithm runs: 86
  Number of successful target algorithm runs: 76
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_m
  Metric: r2
  Best validation score: 0.971285
  Number of target algorithm runs: 96
  Number of successful target algorithm runs: 85
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_m
  Metric: r2
  Best validation score: 0.962499
  Number of target algorithm runs: 79
  Number of successful target algorithm runs: 67
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_m
  Metric: r2
  Best validation score: 0.976071
  Number of target algorithm runs: 84
  Number of successful target algorithm runs: 69
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_cm
  Metric: r2
  Best validation score: 0.916907
  Number of target algorithm runs: 180
  Number of successful target algorithm runs: 164
  Number of crashed target algorithm runs: 15
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_cm
  Metric: r2
  Best validation score: 0.956927
  Number of target algorithm runs: 120
  Number of successful target algorithm runs: 109
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_cm
  Metric: r2
  Best validation score: 0.932065
  Number of target algorithm runs: 116
  Number of successful target algorithm runs: 105
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_cm
  Metric: r2
  Best validation score: 0.974656
  Number of target algorithm runs: 89
  Number of successful target algorithm runs: 75
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_cm
  Metric: r2
  Best validation score: 0.957706
  Number of target algorithm runs: 94
  Number of successful target algorithm runs: 78
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_cm
  Metric: r2
  Best validation score: 0.946713
  Number of target algorithm runs: 69
  Number of successful target algorithm runs: 61
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_cm
  Metric: r2
  Best validation score: 0.985868
  Number of target algorithm runs: 77
  Number of successful target algorithm runs: 65
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_60LH_60IT
  Metric: r2
  Best validation score: 0.701651
  Number of target algorithm runs: 151
  Number of successful target algorithm runs: 142
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_75LH_75IT
  Metric: r2
  Best validation score: 0.897653
  Number of target algorithm runs: 164
  Number of successful target algorithm runs: 149
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'EMU_75LH_75IT', 0.9614363891873168, 3.991330984722971]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [1, 'EMU_75LH_75IT', 0.9614363891873168, 3.991330984722971]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_90LH_90IT
  Metric: r2
  Best validation score: 0.790145
  Number of target algorithm runs: 134
  Number of successful target algorithm runs: 122
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'EMU_90LH_90IT', 0.9086827482148087, 4.058164358690254]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [1, 'EMU_75LH_75IT', 0.9614363891873168, 3.991330984722971], [1, 'EMU_90LH_90IT', 0.9086827482148087, 4.058164358690254]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_110LH_110IT
  Metric: r2
  Best validation score: 0.940787
  Number of target algorithm runs: 102
  Number of successful target algorithm runs: 84
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'EMU_110LH_110IT', 0.9785321593035456, 2.461748193958194]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [1, 'EMU_75LH_75IT', 0.9614363891873168, 3.991330984722971], [1, 'EMU_90LH_90IT', 0.9086827482148087, 4.058164358690254], [1, 'EMU_110LH_110IT', 0.9785321593035456, 2.461748193958194]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_125LH_125IT
  Metric: r2
  Best validation score: 0.890162
  Number of target algorithm runs: 155
  Number of successful target algorithm runs: 142
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'EMU_125LH_125IT', 0.9502625159242173, 3.6779052217807418]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [1, 'EMU_75LH_75IT', 0.9614363891873168, 3.991330984722971], [1, 'EMU_90LH_90IT', 0.9086827482148087, 4.058164358690254], [1, 'EMU_110LH_110IT', 0.9785321593035456, 2.461748193958194], [1, 'EMU_125LH_125IT', 0.9502625159242173, 3.6779052217807418]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_140LH_140IT
  Metric: r2
  Best validation score: 0.926955
  Number of target algorithm runs: 81
  Number of successful target algorithm runs: 72
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'EMU_140LH_140IT', 0.9768662011660407, 2.4990478072566162]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [1, 'EMU_75LH_75IT', 0.9614363891873168, 3.991330984722971], [1, 'EMU_90LH_90IT', 0.9086827482148087, 4.058164358690254], [1, 'EMU_110LH_110IT', 0.9785321593035456, 2.461748193958194], [1, 'EMU_125LH_125IT', 0.9502625159242173, 3.6779052217807418], [1, 'EMU_140LH_140IT', 0.9768662011660407, 2.4990478072566162]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_160LH_160IT
  Metric: r2
  Best validation score: 0.966190
  Number of target algorithm runs: 57
  Number of successful target algorithm runs: 46
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[1, 'EMU_160LH_160IT', 0.9829328955483733, 2.031004438714333]
[[1, 'LH_with_120_c', 0.9477812702630554, 3.2919433936025424], [1, 'LH_with_150_c', 0.9552734740234787, 2.9250798472568764], [1, 'LH_with_180_c', 0.9343070108423822, 3.7430141280494222], [1, 'LH_with_220_c', 0.9699434178568769, 2.09763685126182], [1, 'LH_with_250_c', 0.9781965554225448, 1.7635805680404701], [1, 'LH_with_280_c', 0.9596617410015417, 2.3209150509442327], [1, 'LH_with_320_c', 0.9721444694078851, 2.109578004327688], [1, 'LH_with_120_m', 0.9430841172507985, 3.281575745854854], [1, 'LH_with_150_m', 0.9302378856794684, 3.0831034055773316], [1, 'LH_with_180_m', 0.9590106583472452, 2.3343493360781413], [1, 'LH_with_220_m', 0.9568039411432336, 2.0967144523301466], [1, 'LH_with_250_m', 0.9645082181926379, 2.3006031242543337], [1, 'LH_with_280_m', 0.9434690883331868, 2.4045452719780798], [1, 'LH_with_320_m', 0.9745965157313937, 1.71788751593072], [1, 'LH_with_120_cm', 0.9162891276471453, 4.056866472502521], [1, 'LH_with_150_cm', 0.9557451768959114, 2.6043003446320503], [1, 'LH_with_180_cm', 0.9652768209230047, 2.9811392583104346], [1, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [1, 'LH_with_250_cm', 0.9359606290281927, 2.4040417887696997], [1, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [1, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [1, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [1, 'EMU_75LH_75IT', 0.9614363891873168, 3.991330984722971], [1, 'EMU_90LH_90IT', 0.9086827482148087, 4.058164358690254], [1, 'EMU_110LH_110IT', 0.9785321593035456, 2.461748193958194], [1, 'EMU_125LH_125IT', 0.9502625159242173, 3.6779052217807418], [1, 'EMU_140LH_140IT', 0.9768662011660407, 2.4990478072566162], [1, 'EMU_160LH_160IT', 0.9829328955483733, 2.031004438714333]]

## 6.20.18
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_c
  Metric: r2
  Best validation score: 0.981268
  Number of target algorithm runs: 126
  Number of successful target algorithm runs: 116
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_c
  Metric: r2
  Best validation score: 0.961729
  Number of target algorithm runs: 111
  Number of successful target algorithm runs: 95
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_c
  Metric: r2
  Best validation score: 0.874620
  Number of target algorithm runs: 116
  Number of successful target algorithm runs: 101
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_c
  Metric: r2
  Best validation score: 0.979089
  Number of target algorithm runs: 97
  Number of successful target algorithm runs: 87
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_c
  Metric: r2
  Best validation score: 0.988238
  Number of target algorithm runs: 78
  Number of successful target algorithm runs: 67
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_c
  Metric: r2
  Best validation score: 0.978801
  Number of target algorithm runs: 76
  Number of successful target algorithm runs: 66
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_c
  Metric: r2
  Best validation score: 0.985066
  Number of target algorithm runs: 78
  Number of successful target algorithm runs: 66
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_m
  Metric: r2
  Best validation score: 0.943599
  Number of target algorithm runs: 148
  Number of successful target algorithm runs: 127
  Number of crashed target algorithm runs: 19
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_m
  Metric: r2
  Best validation score: 0.984304
  Number of target algorithm runs: 141
  Number of successful target algorithm runs: 122
  Number of crashed target algorithm runs: 14
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_m
  Metric: r2
  Best validation score: 0.956887
  Number of target algorithm runs: 114
  Number of successful target algorithm runs: 107
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_m
  Metric: r2
  Best validation score: 0.909735
  Number of target algorithm runs: 81
  Number of successful target algorithm runs: 73
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_m
  Metric: r2
  Best validation score: 0.971430
  Number of target algorithm runs: 91
  Number of successful target algorithm runs: 78
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_m
  Metric: r2
  Best validation score: 0.965627
  Number of target algorithm runs: 87
  Number of successful target algorithm runs: 74
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_m
  Metric: r2
  Best validation score: 0.973045
  Number of target algorithm runs: 91
  Number of successful target algorithm runs: 77
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_cm
  Metric: r2
  Best validation score: 0.949980
  Number of target algorithm runs: 189
  Number of successful target algorithm runs: 173
  Number of crashed target algorithm runs: 15
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_cm
  Metric: r2
  Best validation score: 0.957886
  Number of target algorithm runs: 127
  Number of successful target algorithm runs: 116
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_cm
  Metric: r2
  Best validation score: 0.937105
  Number of target algorithm runs: 127
  Number of successful target algorithm runs: 113
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_cm
  Metric: r2
  Best validation score: 0.974656
  Number of target algorithm runs: 89
  Number of successful target algorithm runs: 75
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_cm
  Metric: r2
  Best validation score: 0.957535
  Number of target algorithm runs: 94
  Number of successful target algorithm runs: 81
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_cm
  Metric: r2
  Best validation score: 0.935488
  Number of target algorithm runs: 67
  Number of successful target algorithm runs: 57
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_cm
  Metric: r2
  Best validation score: 0.987582
  Number of target algorithm runs: 75
  Number of successful target algorithm runs: 65
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_60LH_60IT
  Metric: r2
  Best validation score: 0.691350
  Number of target algorithm runs: 143
  Number of successful target algorithm runs: 132
  Number of crashed target algorithm runs: 11
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907], [2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_75LH_75IT
  Metric: r2
  Best validation score: 0.887843
  Number of target algorithm runs: 150
  Number of successful target algorithm runs: 140
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'EMU_75LH_75IT', 0.9470493634695121, 4.41485631996217]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907], [2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [2, 'EMU_75LH_75IT', 0.9470493634695121, 4.41485631996217]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_90LH_90IT
  Metric: r2
  Best validation score: 0.781089
  Number of target algorithm runs: 145
  Number of successful target algorithm runs: 126
  Number of crashed target algorithm runs: 16
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'EMU_90LH_90IT', 0.9512541811048089, 3.168662255863814]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907], [2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [2, 'EMU_75LH_75IT', 0.9470493634695121, 4.41485631996217], [2, 'EMU_90LH_90IT', 0.9512541811048089, 3.168662255863814]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_110LH_110IT
  Metric: r2
  Best validation score: 0.944796
  Number of target algorithm runs: 118
  Number of successful target algorithm runs: 103
  Number of crashed target algorithm runs: 11
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'EMU_110LH_110IT', 0.9762884772419815, 2.3823833315977625]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907], [2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [2, 'EMU_75LH_75IT', 0.9470493634695121, 4.41485631996217], [2, 'EMU_90LH_90IT', 0.9512541811048089, 3.168662255863814], [2, 'EMU_110LH_110IT', 0.9762884772419815, 2.3823833315977625]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_125LH_125IT
  Metric: r2
  Best validation score: 0.890162
  Number of target algorithm runs: 136
  Number of successful target algorithm runs: 124
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'EMU_125LH_125IT', 0.9599866737901258, 3.5531574463983993]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907], [2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [2, 'EMU_75LH_75IT', 0.9470493634695121, 4.41485631996217], [2, 'EMU_90LH_90IT', 0.9512541811048089, 3.168662255863814], [2, 'EMU_110LH_110IT', 0.9762884772419815, 2.3823833315977625], [2, 'EMU_125LH_125IT', 0.9599866737901258, 3.5531574463983993]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_140LH_140IT
  Metric: r2
  Best validation score: 0.907135
  Number of target algorithm runs: 72
  Number of successful target algorithm runs: 64
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'EMU_140LH_140IT', 0.9674333568431834, 3.0477628753450583]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907], [2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [2, 'EMU_75LH_75IT', 0.9470493634695121, 4.41485631996217], [2, 'EMU_90LH_90IT', 0.9512541811048089, 3.168662255863814], [2, 'EMU_110LH_110IT', 0.9762884772419815, 2.3823833315977625], [2, 'EMU_125LH_125IT', 0.9599866737901258, 3.5531574463983993], [2, 'EMU_140LH_140IT', 0.9674333568431834, 3.0477628753450583]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_160LH_160IT
  Metric: r2
  Best validation score: 0.966091
  Number of target algorithm runs: 60
  Number of successful target algorithm runs: 48
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[2, 'EMU_160LH_160IT', 0.9825277195517444, 2.078735509997969]
[[2, 'LH_with_120_c', 0.9476884947057508, 3.2917174018379303], [2, 'LH_with_150_c', 0.9620555788317022, 2.5293708450861034], [2, 'LH_with_180_c', 0.9342692366026992, 3.7125568366599966], [2, 'LH_with_220_c', 0.9733075421883165, 2.006936060402727], [2, 'LH_with_250_c', 0.9817950853548759, 1.5341593475899618], [2, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [2, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [2, 'LH_with_120_m', 0.9311195124750999, 3.5420623822048265], [2, 'LH_with_150_m', 0.9225010236251503, 3.406276543547801], [2, 'LH_with_180_m', 0.9632077923591936, 2.1738392659414156], [2, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [2, 'LH_with_250_m', 0.9612305629459782, 2.4825170584625345], [2, 'LH_with_280_m', 0.9455166906651281, 2.285684188222432], [2, 'LH_with_320_m', 0.9658230039060027, 2.1882536310684233], [2, 'LH_with_120_cm', 0.9210135264024344, 3.8117922462717555], [2, 'LH_with_150_cm', 0.9558586655699568, 2.638345609211071], [2, 'LH_with_180_cm', 0.9665727017044135, 2.7235207487194386], [2, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [2, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [2, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [2, 'LH_with_320_cm', 0.9706381870594055, 1.7263630936361907], [2, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [2, 'EMU_75LH_75IT', 0.9470493634695121, 4.41485631996217], [2, 'EMU_90LH_90IT', 0.9512541811048089, 3.168662255863814], [2, 'EMU_110LH_110IT', 0.9762884772419815, 2.3823833315977625], [2, 'EMU_125LH_125IT', 0.9599866737901258, 3.5531574463983993], [2, 'EMU_140LH_140IT', 0.9674333568431834, 3.0477628753450583], [2, 'EMU_160LH_160IT', 0.9825277195517444, 2.078735509997969]]

## 6.20.23
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_c
  Metric: r2
  Best validation score: 0.980973
  Number of target algorithm runs: 151
  Number of successful target algorithm runs: 132
  Number of crashed target algorithm runs: 16
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_c
  Metric: r2
  Best validation score: 0.958138
  Number of target algorithm runs: 98
  Number of successful target algorithm runs: 86
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_c
  Metric: r2
  Best validation score: 0.874886
  Number of target algorithm runs: 113
  Number of successful target algorithm runs: 100
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_c
  Metric: r2
  Best validation score: 0.980589
  Number of target algorithm runs: 92
  Number of successful target algorithm runs: 80
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_c
  Metric: r2
  Best validation score: 0.978427
  Number of target algorithm runs: 90
  Number of successful target algorithm runs: 77
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_c
  Metric: r2
  Best validation score: 0.985982
  Number of target algorithm runs: 73
  Number of successful target algorithm runs: 64
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_c
  Metric: r2
  Best validation score: 0.985407
  Number of target algorithm runs: 64
  Number of successful target algorithm runs: 52
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_m
  Metric: r2
  Best validation score: 0.936061
  Number of target algorithm runs: 119
  Number of successful target algorithm runs: 105
  Number of crashed target algorithm runs: 11
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_m
  Metric: r2
  Best validation score: 0.981580
  Number of target algorithm runs: 128
  Number of successful target algorithm runs: 115
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_m
  Metric: r2
  Best validation score: 0.954147
  Number of target algorithm runs: 88
  Number of successful target algorithm runs: 72
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_m
  Metric: r2
  Best validation score: 0.912182
  Number of target algorithm runs: 87
  Number of successful target algorithm runs: 75
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_m
  Metric: r2
  Best validation score: 0.971534
  Number of target algorithm runs: 83
  Number of successful target algorithm runs: 72
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_m
  Metric: r2
  Best validation score: 0.975415
  Number of target algorithm runs: 75
  Number of successful target algorithm runs: 61
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_m
  Metric: r2
  Best validation score: 0.969338
  Number of target algorithm runs: 74
  Number of successful target algorithm runs: 63
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_cm
  Metric: r2
  Best validation score: 0.957732
  Number of target algorithm runs: 131
  Number of successful target algorithm runs: 114
  Number of crashed target algorithm runs: 14
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_cm
  Metric: r2
  Best validation score: 0.938626
  Number of target algorithm runs: 126
  Number of successful target algorithm runs: 113
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_cm
  Metric: r2
  Best validation score: 0.934209
  Number of target algorithm runs: 113
  Number of successful target algorithm runs: 102
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_cm
  Metric: r2
  Best validation score: 0.972790
  Number of target algorithm runs: 86
  Number of successful target algorithm runs: 74
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_cm
  Metric: r2
  Best validation score: 0.957535
  Number of target algorithm runs: 68
  Number of successful target algorithm runs: 56
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_cm
  Metric: r2
  Best validation score: 0.963626
  Number of target algorithm runs: 62
  Number of successful target algorithm runs: 52
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_cm
  Metric: r2
  Best validation score: 0.987921
  Number of target algorithm runs: 65
  Number of successful target algorithm runs: 52
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_60LH_60IT
  Metric: r2
  Best validation score: 0.691350
  Number of target algorithm runs: 138
  Number of successful target algorithm runs: 127
  Number of crashed target algorithm runs: 11
  Number of target algorithms that exceeded the time limit: 0
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926], [3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_75LH_75IT
  Metric: r2
  Best validation score: 0.881727
  Number of target algorithm runs: 137
  Number of successful target algorithm runs: 127
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'EMU_75LH_75IT', 0.9264270801700977, 5.176710374970074]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926], [3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [3, 'EMU_75LH_75IT', 0.9264270801700977, 5.176710374970074]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_90LH_90IT
  Metric: r2
  Best validation score: 0.831362
  Number of target algorithm runs: 104
  Number of successful target algorithm runs: 94
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'EMU_90LH_90IT', 0.938715433066432, 3.6914339961934854]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926], [3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [3, 'EMU_75LH_75IT', 0.9264270801700977, 5.176710374970074], [3, 'EMU_90LH_90IT', 0.938715433066432, 3.6914339961934854]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_110LH_110IT
  Metric: r2
  Best validation score: 0.942329
  Number of target algorithm runs: 79
  Number of successful target algorithm runs: 67
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'EMU_110LH_110IT', 0.926750502293858, 4.416188441641392]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926], [3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [3, 'EMU_75LH_75IT', 0.9264270801700977, 5.176710374970074], [3, 'EMU_90LH_90IT', 0.938715433066432, 3.6914339961934854], [3, 'EMU_110LH_110IT', 0.926750502293858, 4.416188441641392]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_125LH_125IT
  Metric: r2
  Best validation score: 0.890162
  Number of target algorithm runs: 111
  Number of successful target algorithm runs: 95
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'EMU_125LH_125IT', 0.9695926852047826, 3.2221415250566494]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926], [3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [3, 'EMU_75LH_75IT', 0.9264270801700977, 5.176710374970074], [3, 'EMU_90LH_90IT', 0.938715433066432, 3.6914339961934854], [3, 'EMU_110LH_110IT', 0.926750502293858, 4.416188441641392], [3, 'EMU_125LH_125IT', 0.9695926852047826, 3.2221415250566494]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_140LH_140IT
  Metric: r2
  Best validation score: 0.918051
  Number of target algorithm runs: 64
  Number of successful target algorithm runs: 55
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'EMU_140LH_140IT', 0.9822440346370168, 3.2019768404789035]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926], [3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [3, 'EMU_75LH_75IT', 0.9264270801700977, 5.176710374970074], [3, 'EMU_90LH_90IT', 0.938715433066432, 3.6914339961934854], [3, 'EMU_110LH_110IT', 0.926750502293858, 4.416188441641392], [3, 'EMU_125LH_125IT', 0.9695926852047826, 3.2221415250566494], [3, 'EMU_140LH_140IT', 0.9822440346370168, 3.2019768404789035]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_160LH_160IT
  Metric: r2
  Best validation score: 0.966720
  Number of target algorithm runs: 60
  Number of successful target algorithm runs: 50
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[3, 'EMU_160LH_160IT', 0.9803949753982442, 2.201758680915845]
[[3, 'LH_with_120_c', 0.9476343351392988, 3.2940595914733697], [3, 'LH_with_150_c', 0.9593961184422723, 2.7759483034229127], [3, 'LH_with_180_c', 0.9319404118157423, 3.749410573486562], [3, 'LH_with_220_c', 0.9741363319692582, 1.8777984865553305], [3, 'LH_with_250_c', 0.9651861698951987, 2.335143222176934], [3, 'LH_with_280_c', 0.9835337478253058, 1.433505794578638], [3, 'LH_with_320_c', 0.9813362966095779, 1.502524891937759], [3, 'LH_with_120_m', 0.9309006420853263, 3.592648324210031], [3, 'LH_with_150_m', 0.9398529835589842, 2.8224970385286317], [3, 'LH_with_180_m', 0.9620975043433033, 2.3190415820046617], [3, 'LH_with_220_m', 0.9545159404528339, 2.5222927172518474], [3, 'LH_with_250_m', 0.9625291405352796, 2.420106010465735], [3, 'LH_with_280_m', 0.9634204272209171, 1.9007879448650928], [3, 'LH_with_320_m', 0.9668048551730111, 2.1860619968245323], [3, 'LH_with_120_cm', 0.9154749539019588, 3.6174346208189387], [3, 'LH_with_150_cm', 0.9582060702440228, 2.5522489790812317], [3, 'LH_with_180_cm', 0.9683663855579122, 2.865938601801463], [3, 'LH_with_220_cm', 0.9616859333451154, 2.426502648129799], [3, 'LH_with_250_cm', 0.9346892356558826, 2.5038558892590728], [3, 'LH_with_280_cm', 0.9875579374536034, 1.5395904836371348], [3, 'LH_with_320_cm', 0.9711650402112932, 1.7545609990469926], [3, 'EMU_60LH_60IT', 0.9353150974128626, 4.209216589727868], [3, 'EMU_75LH_75IT', 0.9264270801700977, 5.176710374970074], [3, 'EMU_90LH_90IT', 0.938715433066432, 3.6914339961934854], [3, 'EMU_110LH_110IT', 0.926750502293858, 4.416188441641392], [3, 'EMU_125LH_125IT', 0.9695926852047826, 3.2221415250566494], [3, 'EMU_140LH_140IT', 0.9822440346370168, 3.2019768404789035], [3, 'EMU_160LH_160IT', 0.9803949753982442, 2.201758680915845]]

## 6.21.11
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_c
  Metric: r2
  Best validation score: 0.958247
  Number of target algorithm runs: 166
  Number of successful target algorithm runs: 152
  Number of crashed target algorithm runs: 13
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_c
  Metric: r2
  Best validation score: 0.964129
  Number of target algorithm runs: 119
  Number of successful target algorithm runs: 110
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_c
  Metric: r2
  Best validation score: 0.884147
  Number of target algorithm runs: 111
  Number of successful target algorithm runs: 102
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_c
  Metric: r2
  Best validation score: 0.975432
  Number of target algorithm runs: 96
  Number of successful target algorithm runs: 84
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_c
  Metric: r2
  Best validation score: 0.978998
  Number of target algorithm runs: 95
  Number of successful target algorithm runs: 85
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_c
  Metric: r2
  Best validation score: 0.978801
  Number of target algorithm runs: 77
  Number of successful target algorithm runs: 66
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_c
  Metric: r2
  Best validation score: 0.981294
  Number of target algorithm runs: 74
  Number of successful target algorithm runs: 64
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_m
  Metric: r2
  Best validation score: 0.949944
  Number of target algorithm runs: 163
  Number of successful target algorithm runs: 139
  Number of crashed target algorithm runs: 20
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_m
  Metric: r2
  Best validation score: 0.983392
  Number of target algorithm runs: 137
  Number of successful target algorithm runs: 122
  Number of crashed target algorithm runs: 13
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_m
  Metric: r2
  Best validation score: 0.956906
  Number of target algorithm runs: 127
  Number of successful target algorithm runs: 120
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_m
  Metric: r2
  Best validation score: 0.909735
  Number of target algorithm runs: 81
  Number of successful target algorithm runs: 73
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_m
  Metric: r2
  Best validation score: 0.974246
  Number of target algorithm runs: 83
  Number of successful target algorithm runs: 70
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_m
  Metric: r2
  Best validation score: 0.964751
  Number of target algorithm runs: 90
  Number of successful target algorithm runs: 78
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_m
  Metric: r2
  Best validation score: 0.970283
  Number of target algorithm runs: 87
  Number of successful target algorithm runs: 78
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_cm
  Metric: r2
  Best validation score: 0.949981
  Number of target algorithm runs: 174
  Number of successful target algorithm runs: 158
  Number of crashed target algorithm runs: 15
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_cm
  Metric: r2
  Best validation score: 0.966993
  Number of target algorithm runs: 136
  Number of successful target algorithm runs: 127
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_cm
  Metric: r2
  Best validation score: 0.934192
  Number of target algorithm runs: 135
  Number of successful target algorithm runs: 125
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_cm
  Metric: r2
  Best validation score: 0.976017
  Number of target algorithm runs: 99
  Number of successful target algorithm runs: 83
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_cm
  Metric: r2
  Best validation score: 0.957535
  Number of target algorithm runs: 89
  Number of successful target algorithm runs: 77
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_cm
  Metric: r2
  Best validation score: 0.953733
  Number of target algorithm runs: 95
  Number of successful target algorithm runs: 82
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_cm
  Metric: r2
  Best validation score: 0.985868
  Number of target algorithm runs: 76
  Number of successful target algorithm runs: 64
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_60LH_60IT
  Metric: r2
  Best validation score: 0.701651
  Number of target algorithm runs: 151
  Number of successful target algorithm runs: 142
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_75LH_75IT
  Metric: r2
  Best validation score: 0.912271
  Number of target algorithm runs: 160
  Number of successful target algorithm runs: 147
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'EMU_75LH_75IT', 0.9579149959585942, 3.5445430387029804]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [4, 'EMU_75LH_75IT', 0.9579149959585942, 3.5445430387029804]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_90LH_90IT
  Metric: r2
  Best validation score: 0.804020
  Number of target algorithm runs: 139
  Number of successful target algorithm runs: 120
  Number of crashed target algorithm runs: 16
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [4, 'EMU_75LH_75IT', 0.9579149959585942, 3.5445430387029804], [4, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_110LH_110IT
  Metric: r2
  Best validation score: 0.944417
  Number of target algorithm runs: 116
  Number of successful target algorithm runs: 104
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'EMU_110LH_110IT', 0.9773472175259671, 2.4366761039051235]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [4, 'EMU_75LH_75IT', 0.9579149959585942, 3.5445430387029804], [4, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [4, 'EMU_110LH_110IT', 0.9773472175259671, 2.4366761039051235]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_125LH_125IT
  Metric: r2
  Best validation score: 0.890162
  Number of target algorithm runs: 139
  Number of successful target algorithm runs: 127
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'EMU_125LH_125IT', 0.9657377496052835, 3.5400622786033864]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [4, 'EMU_75LH_75IT', 0.9579149959585942, 3.5445430387029804], [4, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [4, 'EMU_110LH_110IT', 0.9773472175259671, 2.4366761039051235], [4, 'EMU_125LH_125IT', 0.9657377496052835, 3.5400622786033864]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_140LH_140IT
  Metric: r2
  Best validation score: 0.912175
  Number of target algorithm runs: 90
  Number of successful target algorithm runs: 80
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'EMU_140LH_140IT', 0.9782168782185602, 2.616431921028]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [4, 'EMU_75LH_75IT', 0.9579149959585942, 3.5445430387029804], [4, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [4, 'EMU_110LH_110IT', 0.9773472175259671, 2.4366761039051235], [4, 'EMU_125LH_125IT', 0.9657377496052835, 3.5400622786033864], [4, 'EMU_140LH_140IT', 0.9782168782185602, 2.616431921028]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_160LH_160IT
  Metric: r2
  Best validation score: 0.967735
  Number of target algorithm runs: 59
  Number of successful target algorithm runs: 46
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 10
  Number of target algorithms that exceeded the memory limit: 0

saved
[4, 'EMU_160LH_160IT', 0.9827080430102808, 2.0981674948910327]
[[4, 'LH_with_120_c', 0.9500976570951288, 3.625655982667816], [4, 'LH_with_150_c', 0.9641100935598075, 2.4888650253839866], [4, 'LH_with_180_c', 0.9375472132024226, 3.3955500400088314], [4, 'LH_with_220_c', 0.970268491625301, 2.1846004785619555], [4, 'LH_with_250_c', 0.9634432979827902, 2.4317458023975393], [4, 'LH_with_280_c', 0.9727164236489858, 1.9071336638937681], [4, 'LH_with_320_c', 0.9559826881484608, 2.2260243911119244], [4, 'LH_with_120_m', 0.9442992048130627, 3.0371267894039855], [4, 'LH_with_150_m', 0.9433965620777902, 2.7838394817404555], [4, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [4, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [4, 'LH_with_250_m', 0.9651327307217427, 2.3010556833806417], [4, 'LH_with_280_m', 0.9495724722455245, 2.24584469663785], [4, 'LH_with_320_m', 0.9621106771769796, 2.360268276064712], [4, 'LH_with_120_cm', 0.921998629952662, 3.6958259745699484], [4, 'LH_with_150_cm', 0.9548572571188078, 2.650595346603289], [4, 'LH_with_180_cm', 0.9711344893235302, 2.696598553505549], [4, 'LH_with_220_cm', 0.9649776845941843, 2.344142045502444], [4, 'LH_with_250_cm', 0.9501042037910494, 2.1965946659712356], [4, 'LH_with_280_cm', 0.9865740976691574, 1.5466757908542668], [4, 'LH_with_320_cm', 0.9681785396997785, 1.908894467767952], [4, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [4, 'EMU_75LH_75IT', 0.9579149959585942, 3.5445430387029804], [4, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [4, 'EMU_110LH_110IT', 0.9773472175259671, 2.4366761039051235], [4, 'EMU_125LH_125IT', 0.9657377496052835, 3.5400622786033864], [4, 'EMU_140LH_140IT', 0.9782168782185602, 2.616431921028], [4, 'EMU_160LH_160IT', 0.9827080430102808, 2.0981674948910327]]

## 6.21.18
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_c
  Metric: r2
  Best validation score: 0.960257
  Number of target algorithm runs: 142
  Number of successful target algorithm runs: 128
  Number of crashed target algorithm runs: 11
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_c
  Metric: r2
  Best validation score: 0.960304
  Number of target algorithm runs: 92
  Number of successful target algorithm runs: 81
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_c
  Metric: r2
  Best validation score: 0.874392
  Number of target algorithm runs: 107
  Number of successful target algorithm runs: 93
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_c
  Metric: r2
  Best validation score: 0.966122
  Number of target algorithm runs: 101
  Number of successful target algorithm runs: 93
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_c
  Metric: r2
  Best validation score: 0.987820
  Number of target algorithm runs: 112
  Number of successful target algorithm runs: 97
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_c
  Metric: r2
  Best validation score: 0.969654
  Number of target algorithm runs: 75
  Number of successful target algorithm runs: 65
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_c
  Metric: r2
  Best validation score: 0.985066
  Number of target algorithm runs: 79
  Number of successful target algorithm runs: 67
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_m
  Metric: r2
  Best validation score: 0.950056
  Number of target algorithm runs: 175
  Number of successful target algorithm runs: 155
  Number of crashed target algorithm runs: 17
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_m
  Metric: r2
  Best validation score: 0.983732
  Number of target algorithm runs: 138
  Number of successful target algorithm runs: 127
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_m
  Metric: r2
  Best validation score: 0.953109
  Number of target algorithm runs: 115
  Number of successful target algorithm runs: 107
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_m
  Metric: r2
  Best validation score: 0.909735
  Number of target algorithm runs: 81
  Number of successful target algorithm runs: 73
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_m
  Metric: r2
  Best validation score: 0.971961
  Number of target algorithm runs: 97
  Number of successful target algorithm runs: 80
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_m
  Metric: r2
  Best validation score: 0.970768
  Number of target algorithm runs: 90
  Number of successful target algorithm runs: 77
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_m
  Metric: r2
  Best validation score: 0.970283
  Number of target algorithm runs: 81
  Number of successful target algorithm runs: 69
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_cm
  Metric: r2
  Best validation score: 0.957052
  Number of target algorithm runs: 162
  Number of successful target algorithm runs: 142
  Number of crashed target algorithm runs: 17
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_cm
  Metric: r2
  Best validation score: 0.955315
  Number of target algorithm runs: 127
  Number of successful target algorithm runs: 117
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_cm
  Metric: r2
  Best validation score: 0.932651
  Number of target algorithm runs: 115
  Number of successful target algorithm runs: 104
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_cm
  Metric: r2
  Best validation score: 0.974656
  Number of target algorithm runs: 89
  Number of successful target algorithm runs: 75
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_cm
  Metric: r2
  Best validation score: 0.957535
  Number of target algorithm runs: 95
  Number of successful target algorithm runs: 82
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_cm
  Metric: r2
  Best validation score: 0.935488
  Number of target algorithm runs: 69
  Number of successful target algorithm runs: 59
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_cm
  Metric: r2
  Best validation score: 0.979884
  Number of target algorithm runs: 85
  Number of successful target algorithm runs: 71
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_60LH_60IT
  Metric: r2
  Best validation score: 0.691350
  Number of target algorithm runs: 157
  Number of successful target algorithm runs: 144
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128], [5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_75LH_75IT
  Metric: r2
  Best validation score: 0.905980
  Number of target algorithm runs: 168
  Number of successful target algorithm runs: 157
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'EMU_75LH_75IT', 0.9460356697621436, 4.298573007059038]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128], [5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942], [5, 'EMU_75LH_75IT', 0.9460356697621436, 4.298573007059038]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_90LH_90IT
  Metric: r2
  Best validation score: 0.781089
  Number of target algorithm runs: 140
  Number of successful target algorithm runs: 122
  Number of crashed target algorithm runs: 15
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128], [5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942], [5, 'EMU_75LH_75IT', 0.9460356697621436, 4.298573007059038], [5, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_110LH_110IT
  Metric: r2
  Best validation score: 0.944796
  Number of target algorithm runs: 111
  Number of successful target algorithm runs: 98
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'EMU_110LH_110IT', 0.9763977816976108, 2.5799537499361613]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128], [5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942], [5, 'EMU_75LH_75IT', 0.9460356697621436, 4.298573007059038], [5, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [5, 'EMU_110LH_110IT', 0.9763977816976108, 2.5799537499361613]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_125LH_125IT
  Metric: r2
  Best validation score: 0.890162
  Number of target algorithm runs: 146
  Number of successful target algorithm runs: 125
  Number of crashed target algorithm runs: 18
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'EMU_125LH_125IT', 0.9401918068357931, 4.820442677433039]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128], [5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942], [5, 'EMU_75LH_75IT', 0.9460356697621436, 4.298573007059038], [5, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [5, 'EMU_110LH_110IT', 0.9763977816976108, 2.5799537499361613], [5, 'EMU_125LH_125IT', 0.9401918068357931, 4.820442677433039]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_140LH_140IT
  Metric: r2
  Best validation score: 0.887382
  Number of target algorithm runs: 118
  Number of successful target algorithm runs: 108
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'EMU_140LH_140IT', 0.9703820983077666, 2.9570100232916463]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128], [5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942], [5, 'EMU_75LH_75IT', 0.9460356697621436, 4.298573007059038], [5, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [5, 'EMU_110LH_110IT', 0.9763977816976108, 2.5799537499361613], [5, 'EMU_125LH_125IT', 0.9401918068357931, 4.820442677433039], [5, 'EMU_140LH_140IT', 0.9703820983077666, 2.9570100232916463]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_160LH_160IT
  Metric: r2
  Best validation score: 0.968233
  Number of target algorithm runs: 64
  Number of successful target algorithm runs: 51
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[5, 'EMU_160LH_160IT', 0.982729168320804, 2.1001327394165243]
[[5, 'LH_with_120_c', 0.9502205404221972, 3.667787189265079], [5, 'LH_with_150_c', 0.9616678611105938, 2.5259653176716323], [5, 'LH_with_180_c', 0.9182348384632846, 3.886855893847093], [5, 'LH_with_220_c', 0.957955676815058, 2.7252409490730845], [5, 'LH_with_250_c', 0.982392244395097, 1.4802379219938648], [5, 'LH_with_280_c', 0.9593299989022971, 2.353594873211444], [5, 'LH_with_320_c', 0.9603793642459051, 2.0325725603178633], [5, 'LH_with_120_m', 0.9496840267225365, 3.0089327628288074], [5, 'LH_with_150_m', 0.9262228836729445, 3.215728059006935], [5, 'LH_with_180_m', 0.9605351328004701, 2.348271678597377], [5, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [5, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [5, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [5, 'LH_with_320_m', 0.963954625671779, 2.2988866576828952], [5, 'LH_with_120_cm', 0.9287467897169455, 3.4470208892343654], [5, 'LH_with_150_cm', 0.9576152179763585, 2.495684718713751], [5, 'LH_with_180_cm', 0.9670155638263995, 2.7805142812462553], [5, 'LH_with_220_cm', 0.9629433002853877, 2.296146643861614], [5, 'LH_with_250_cm', 0.9373480444235879, 2.4893433460742354], [5, 'LH_with_280_cm', 0.9808714180146062, 2.1080434494821922], [5, 'LH_with_320_cm', 0.966976982094555, 1.9125343786868128], [5, 'EMU_60LH_60IT', 0.9333048269890944, 4.21011255174942], [5, 'EMU_75LH_75IT', 0.9460356697621436, 4.298573007059038], [5, 'EMU_90LH_90IT', 0.947378654586628, 3.2256747057267443], [5, 'EMU_110LH_110IT', 0.9763977816976108, 2.5799537499361613], [5, 'EMU_125LH_125IT', 0.9401918068357931, 4.820442677433039], [5, 'EMU_140LH_140IT', 0.9703820983077666, 2.9570100232916463], [5, 'EMU_160LH_160IT', 0.982729168320804, 2.1001327394165243]]

## 6.22.0
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_c
  Metric: r2
  Best validation score: 0.963974
  Number of target algorithm runs: 144
  Number of successful target algorithm runs: 133
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_c
  Metric: r2
  Best validation score: 0.964066
  Number of target algorithm runs: 129
  Number of successful target algorithm runs: 115
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_c
  Metric: r2
  Best validation score: 0.874392
  Number of target algorithm runs: 112
  Number of successful target algorithm runs: 102
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_c
  Metric: r2
  Best validation score: 0.966258
  Number of target algorithm runs: 101
  Number of successful target algorithm runs: 88
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_c
  Metric: r2
  Best validation score: 0.986934
  Number of target algorithm runs: 84
  Number of successful target algorithm runs: 75
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_c
  Metric: r2
  Best validation score: 0.965972
  Number of target algorithm runs: 78
  Number of successful target algorithm runs: 70
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_c
  Metric: r2
  Best validation score: 0.985033
  Number of target algorithm runs: 67
  Number of successful target algorithm runs: 58
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_m
  Metric: r2
  Best validation score: 0.945411
  Number of target algorithm runs: 173
  Number of successful target algorithm runs: 155
  Number of crashed target algorithm runs: 16
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_m
  Metric: r2
  Best validation score: 0.983392
  Number of target algorithm runs: 136
  Number of successful target algorithm runs: 120
  Number of crashed target algorithm runs: 13
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_m
  Metric: r2
  Best validation score: 0.956906
  Number of target algorithm runs: 127
  Number of successful target algorithm runs: 119
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_m
  Metric: r2
  Best validation score: 0.909735
  Number of target algorithm runs: 81
  Number of successful target algorithm runs: 73
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_m
  Metric: r2
  Best validation score: 0.971961
  Number of target algorithm runs: 97
  Number of successful target algorithm runs: 79
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_m
  Metric: r2
  Best validation score: 0.970768
  Number of target algorithm runs: 90
  Number of successful target algorithm runs: 77
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_m
  Metric: r2
  Best validation score: 0.973309
  Number of target algorithm runs: 83
  Number of successful target algorithm runs: 67
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_cm
  Metric: r2
  Best validation score: 0.952086
  Number of target algorithm runs: 194
  Number of successful target algorithm runs: 174
  Number of crashed target algorithm runs: 17
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_cm
  Metric: r2
  Best validation score: 0.960569
  Number of target algorithm runs: 126
  Number of successful target algorithm runs: 116
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_cm
  Metric: r2
  Best validation score: 0.937899
  Number of target algorithm runs: 124
  Number of successful target algorithm runs: 112
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_cm
  Metric: r2
  Best validation score: 0.974401
  Number of target algorithm runs: 98
  Number of successful target algorithm runs: 86
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_cm
  Metric: r2
  Best validation score: 0.967191
  Number of target algorithm runs: 103
  Number of successful target algorithm runs: 89
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_cm
  Metric: r2
  Best validation score: 0.946713
  Number of target algorithm runs: 70
  Number of successful target algorithm runs: 61
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_cm
  Metric: r2
  Best validation score: 0.977847
  Number of target algorithm runs: 78
  Number of successful target algorithm runs: 65
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_60LH_60IT
  Metric: r2
  Best validation score: 0.701651
  Number of target algorithm runs: 154
  Number of successful target algorithm runs: 144
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094], [6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_75LH_75IT
  Metric: r2
  Best validation score: 0.888905
  Number of target algorithm runs: 177
  Number of successful target algorithm runs: 162
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_75LH_75IT', 0.9497494938279291, 4.335686880756131]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094], [6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [6, 'EMU_75LH_75IT', 0.9497494938279291, 4.335686880756131]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_90LH_90IT
  Metric: r2
  Best validation score: 0.781089
  Number of target algorithm runs: 158
  Number of successful target algorithm runs: 138
  Number of crashed target algorithm runs: 19
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_90LH_90IT', 0.9539859931176606, 2.8572797676936474]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094], [6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [6, 'EMU_75LH_75IT', 0.9497494938279291, 4.335686880756131], [6, 'EMU_90LH_90IT', 0.9539859931176606, 2.8572797676936474]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_110LH_110IT
  Metric: r2
  Best validation score: 0.942826
  Number of target algorithm runs: 78
  Number of successful target algorithm runs: 64
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_110LH_110IT', 0.9681365103514707, 3.0687024698496033]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094], [6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [6, 'EMU_75LH_75IT', 0.9497494938279291, 4.335686880756131], [6, 'EMU_90LH_90IT', 0.9539859931176606, 2.8572797676936474], [6, 'EMU_110LH_110IT', 0.9681365103514707, 3.0687024698496033]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_125LH_125IT
  Metric: r2
  Best validation score: 0.890162
  Number of target algorithm runs: 142
  Number of successful target algorithm runs: 131
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_125LH_125IT', 0.9599287531819021, 3.5558466653652174]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094], [6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [6, 'EMU_75LH_75IT', 0.9497494938279291, 4.335686880756131], [6, 'EMU_90LH_90IT', 0.9539859931176606, 2.8572797676936474], [6, 'EMU_110LH_110IT', 0.9681365103514707, 3.0687024698496033], [6, 'EMU_125LH_125IT', 0.9599287531819021, 3.5558466653652174]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_140LH_140IT
  Metric: r2
  Best validation score: 0.891101
  Number of target algorithm runs: 90
  Number of successful target algorithm runs: 76
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_140LH_140IT', 0.9689467676874371, 3.1958465672506557]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094], [6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [6, 'EMU_75LH_75IT', 0.9497494938279291, 4.335686880756131], [6, 'EMU_90LH_90IT', 0.9539859931176606, 2.8572797676936474], [6, 'EMU_110LH_110IT', 0.9681365103514707, 3.0687024698496033], [6, 'EMU_125LH_125IT', 0.9599287531819021, 3.5558466653652174], [6, 'EMU_140LH_140IT', 0.9689467676874371, 3.1958465672506557]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_160LH_160IT
  Metric: r2
  Best validation score: 0.966201
  Number of target algorithm runs: 52
  Number of successful target algorithm runs: 40
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_160LH_160IT', 0.9817073621988832, 2.115638793086203]
[[6, 'LH_with_120_c', 0.9439019112684726, 3.5625746181853137], [6, 'LH_with_150_c', 0.9549272361530198, 2.855440866208425], [6, 'LH_with_180_c', 0.9330585505245446, 3.700524868542215], [6, 'LH_with_220_c', 0.9577855743997014, 2.7646721170199533], [6, 'LH_with_250_c', 0.9768647654667123, 1.8570574835498708], [6, 'LH_with_280_c', 0.962187774400716, 2.2780854914572184], [6, 'LH_with_320_c', 0.9807740812622731, 1.5140701082806263], [6, 'LH_with_120_m', 0.9400308825458373, 3.2426160142741978], [6, 'LH_with_150_m', 0.9416329609285075, 2.8802721831846108], [6, 'LH_with_180_m', 0.9589014898171253, 2.2445530187770264], [6, 'LH_with_220_m', 0.9544414170267324, 2.347833814382793], [6, 'LH_with_250_m', 0.9699901887500121, 2.0175930434957032], [6, 'LH_with_280_m', 0.9592561444705623, 2.184351960675293], [6, 'LH_with_320_m', 0.96587863343206, 2.17691071803259], [6, 'LH_with_120_cm', 0.9242724899237605, 3.5724626357079368], [6, 'LH_with_150_cm', 0.9564338160959792, 2.6314588945470168], [6, 'LH_with_180_cm', 0.964501741462773, 2.778248266970394], [6, 'LH_with_220_cm', 0.9645782281347497, 2.2666203399983917], [6, 'LH_with_250_cm', 0.9584275133379954, 2.00757268267923], [6, 'LH_with_280_cm', 0.9850228617948873, 1.7178822085950487], [6, 'LH_with_320_cm', 0.9628176810810362, 2.1607464093279094], [6, 'EMU_60LH_60IT', 0.9369894524707856, 4.059410472648899], [6, 'EMU_75LH_75IT', 0.9497494938279291, 4.335686880756131], [6, 'EMU_90LH_90IT', 0.9539859931176606, 2.8572797676936474], [6, 'EMU_110LH_110IT', 0.9681365103514707, 3.0687024698496033], [6, 'EMU_125LH_125IT', 0.9599287531819021, 3.5558466653652174], [6, 'EMU_140LH_140IT', 0.9689467676874371, 3.1958465672506557], [6, 'EMU_160LH_160IT', 0.9817073621988832, 2.115638793086203]]

## 6.22.13
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_c
  Metric: r2
  Best validation score: 0.965721
  Number of target algorithm runs: 145
  Number of successful target algorithm runs: 131
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_c
  Metric: r2
  Best validation score: 0.964644
  Number of target algorithm runs: 109
  Number of successful target algorithm runs: 99
  Number of crashed target algorithm runs: 8
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_c
  Metric: r2
  Best validation score: 0.874392
  Number of target algorithm runs: 102
  Number of successful target algorithm runs: 90
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_c
  Metric: r2
  Best validation score: 0.966122
  Number of target algorithm runs: 93
  Number of successful target algorithm runs: 83
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_c
  Metric: r2
  Best validation score: 0.986757
  Number of target algorithm runs: 75
  Number of successful target algorithm runs: 62
  Number of crashed target algorithm runs: 5
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_c
  Metric: r2
  Best validation score: 0.969020
  Number of target algorithm runs: 72
  Number of successful target algorithm runs: 62
  Number of crashed target algorithm runs: 3
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_c
  Metric: r2
  Best validation score: 0.985703
  Number of target algorithm runs: 65
  Number of successful target algorithm runs: 54
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 9
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_m
  Metric: r2
  Best validation score: 0.942037
  Number of target algorithm runs: 116
  Number of successful target algorithm runs: 105
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_m
  Metric: r2
  Best validation score: 0.983733
  Number of target algorithm runs: 113
  Number of successful target algorithm runs: 101
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_m
  Metric: r2
  Best validation score: 0.955179
  Number of target algorithm runs: 101
  Number of successful target algorithm runs: 91
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_m
  Metric: r2
  Best validation score: 0.926283
  Number of target algorithm runs: 102
  Number of successful target algorithm runs: 92
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_m
  Metric: r2
  Best validation score: 0.971122
  Number of target algorithm runs: 82
  Number of successful target algorithm runs: 72
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_m
  Metric: r2
  Best validation score: 0.975420
  Number of target algorithm runs: 80
  Number of successful target algorithm runs: 71
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_m
  Metric: r2
  Best validation score: 0.969280
  Number of target algorithm runs: 74
  Number of successful target algorithm runs: 59
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_120_cm
  Metric: r2
  Best validation score: 0.957242
  Number of target algorithm runs: 127
  Number of successful target algorithm runs: 114
  Number of crashed target algorithm runs: 12
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_150_cm
  Metric: r2
  Best validation score: 0.957345
  Number of target algorithm runs: 104
  Number of successful target algorithm runs: 94
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 3
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_180_cm
  Metric: r2
  Best validation score: 0.934155
  Number of target algorithm runs: 114
  Number of successful target algorithm runs: 99
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_220_cm
  Metric: r2
  Best validation score: 0.974888
  Number of target algorithm runs: 99
  Number of successful target algorithm runs: 85
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_250_cm
  Metric: r2
  Best validation score: 0.957535
  Number of target algorithm runs: 80
  Number of successful target algorithm runs: 70
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_280_cm
  Metric: r2
  Best validation score: 0.943887
  Number of target algorithm runs: 60
  Number of successful target algorithm runs: 52
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015]]
training...
training successfully
auto-sklearn results:
  Dataset name: LH_with_320_cm
  Metric: r2
  Best validation score: 0.983237
  Number of target algorithm runs: 85
  Number of successful target algorithm runs: 72
  Number of crashed target algorithm runs: 7
  Number of target algorithms that exceeded the time limit: 6
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_60LH_60IT
  Metric: r2
  Best validation score: 0.691350
  Number of target algorithm runs: 144
  Number of successful target algorithm runs: 134
  Number of crashed target algorithm runs: 9
  Number of target algorithms that exceeded the time limit: 1
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712], [6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_75LH_75IT
  Metric: r2
  Best validation score: 0.881727
  Number of target algorithm runs: 153
  Number of successful target algorithm runs: 140
  Number of crashed target algorithm runs: 11
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_75LH_75IT', 0.9352766232732277, 4.877218860731308]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712], [6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866], [6, 'EMU_75LH_75IT', 0.9352766232732277, 4.877218860731308]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_90LH_90IT
  Metric: r2
  Best validation score: 0.841140
  Number of target algorithm runs: 101
  Number of successful target algorithm runs: 87
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_90LH_90IT', 0.920535718518647, 3.7384139551829203]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712], [6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866], [6, 'EMU_75LH_75IT', 0.9352766232732277, 4.877218860731308], [6, 'EMU_90LH_90IT', 0.920535718518647, 3.7384139551829203]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_110LH_110IT
  Metric: r2
  Best validation score: 0.948328
  Number of target algorithm runs: 88
  Number of successful target algorithm runs: 71
  Number of crashed target algorithm runs: 10
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_110LH_110IT', 0.9737726642784584, 2.8869730352316285]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712], [6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866], [6, 'EMU_75LH_75IT', 0.9352766232732277, 4.877218860731308], [6, 'EMU_90LH_90IT', 0.920535718518647, 3.7384139551829203], [6, 'EMU_110LH_110IT', 0.9737726642784584, 2.8869730352316285]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_125LH_125IT
  Metric: r2
  Best validation score: 0.913153
  Number of target algorithm runs: 128
  Number of successful target algorithm runs: 120
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_125LH_125IT', 0.9051565752363492, 4.163608467251225]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712], [6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866], [6, 'EMU_75LH_75IT', 0.9352766232732277, 4.877218860731308], [6, 'EMU_90LH_90IT', 0.920535718518647, 3.7384139551829203], [6, 'EMU_110LH_110IT', 0.9737726642784584, 2.8869730352316285], [6, 'EMU_125LH_125IT', 0.9051565752363492, 4.163608467251225]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_140LH_140IT
  Metric: r2
  Best validation score: 0.898416
  Number of target algorithm runs: 86
  Number of successful target algorithm runs: 77
  Number of crashed target algorithm runs: 4
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_140LH_140IT', 0.9809470404891629, 2.7367497320415906]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712], [6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866], [6, 'EMU_75LH_75IT', 0.9352766232732277, 4.877218860731308], [6, 'EMU_90LH_90IT', 0.920535718518647, 3.7384139551829203], [6, 'EMU_110LH_110IT', 0.9737726642784584, 2.8869730352316285], [6, 'EMU_125LH_125IT', 0.9051565752363492, 4.163608467251225], [6, 'EMU_140LH_140IT', 0.9809470404891629, 2.7367497320415906]]
training...
training successfully
auto-sklearn results:
  Dataset name: EMU_160LH_160IT
  Metric: r2
  Best validation score: 0.966657
  Number of target algorithm runs: 70
  Number of successful target algorithm runs: 56
  Number of crashed target algorithm runs: 6
  Number of target algorithms that exceeded the time limit: 8
  Number of target algorithms that exceeded the memory limit: 0

saved
[6, 'EMU_160LH_160IT', 0.980374919649276, 2.202746914404119]
[[6, 'LH_with_120_c', 0.9477990687966877, 3.3406436571991325], [6, 'LH_with_150_c', 0.9631448026207166, 2.5141392142079524], [6, 'LH_with_180_c', 0.9334283109206774, 3.704901819674812], [6, 'LH_with_220_c', 0.9566748838283493, 2.821189838505384], [6, 'LH_with_250_c', 0.9797782652039875, 1.646055776031461], [6, 'LH_with_280_c', 0.9643728167477235, 2.234711015126249], [6, 'LH_with_320_c', 0.9816387685952512, 1.4837466686060188], [6, 'LH_with_120_m', 0.9285432094135974, 3.428354285964828], [6, 'LH_with_150_m', 0.9277200627556454, 3.2086491351753357], [6, 'LH_with_180_m', 0.9556922192886609, 2.621817583314739], [6, 'LH_with_220_m', 0.969064165557289, 1.6959542162909802], [6, 'LH_with_250_m', 0.9641085615295636, 2.339017880198897], [6, 'LH_with_280_m', 0.962102014558002, 1.9468294756045377], [6, 'LH_with_320_m', 0.9717944714837056, 1.7748999377024257], [6, 'LH_with_120_cm', 0.9210224629066333, 3.5594475360573443], [6, 'LH_with_150_cm', 0.9566736477342468, 2.624520765194111], [6, 'LH_with_180_cm', 0.9626066140192083, 3.1567676517753105], [6, 'LH_with_220_cm', 0.9629647558302509, 2.428971718254641], [6, 'LH_with_250_cm', 0.9402025392255595, 2.381076373258128], [6, 'LH_with_280_cm', 0.9801294471535759, 2.20634362701015], [6, 'LH_with_320_cm', 0.9640731292407255, 2.1546543555136712], [6, 'EMU_60LH_60IT', 0.9384199746290386, 3.9971869527449866], [6, 'EMU_75LH_75IT', 0.9352766232732277, 4.877218860731308], [6, 'EMU_90LH_90IT', 0.920535718518647, 3.7384139551829203], [6, 'EMU_110LH_110IT', 0.9737726642784584, 2.8869730352316285], [6, 'EMU_125LH_125IT', 0.9051565752363492, 4.163608467251225], [6, 'EMU_140LH_140IT', 0.9809470404891629, 2.7367497320415906], [6, 'EMU_160LH_160IT', 0.980374919649276, 2.202746914404119]]